In [ ]:
accountkey = dbutils.secrets.get('databricks-scope','storageaccount-access-key')

In [ ]:
dbutils.fs.mounts()

In [ ]:
already_mounted = False 
for x in dbutils.fs.mounts():
    if x.mountPoint == "/mnt/raw-data":
        already_mounted = True
        print("It is already mounted")
        break
    else:
        already_mounted = False
if not already_mounted:
    dbutils.fs.mount(source = 'wasbs://raw-data@hyderabaddtorageaccount.blob.core.windows.net',
                     mount_point = '/mnt/raw-data',
                     extra_configs={'fs.azure.account.key.hyderabaddtorageaccount.blob.core.windows.net':accountkey})
    print("Mount Created")

It is already mounted


In [ ]:
already_mounted = False 
for x in dbutils.fs.mounts():
    if x.mountPoint == "/mnt/transformed-data":
        already_mounted = True
        print("It is already mounted")
        break
    else:
        already_mounted = False
if not already_mounted:
    dbutils.fs.mount(source = 'wasbs://transformed-data@hyderabaddtorageaccount.blob.core.windows.net',
                     mount_point = '/mnt/transformed-data',
                     extra_configs={'fs.azure.account.key.hyderabaddtorageaccount.blob.core.windows.net':accountkey})
    print("Mount Created")

It is already mounted


In [ ]:
df = spark.read.csv("/mnt/raw-data/walmart/customer/customer.csv", inferSchema=True,header=True)
df.printSchema()

In [ ]:
from pyspark.sql.functions import*

column = df.columns

for col_name in column:
        if "Date" in col_name or "date" in col_name:
            df = df.withColumn(("modified_"+col_name), to_date(col(col_name)))

df.printSchema()
display(df)

In [ ]:
table_name = []
for i in dbutils.fs.ls('/mnt/raw-data/walmart/'):
    table_name.append(i.name.split('/')[0])
table_name

['block',
 'buys',
 'category',
 'customer',
 'customertype',
 'employee',
 'feedback',
 'invoicedetails',
 'offerdetails',
 'paymentmode',
 'product',
 'returnslip',
 'warehouse']

In [ ]:
for i in table_name:
    path = '/mnt/raw-data/walmart/' + i + '/' + i +  '.csv'
    df = spark.read.option("header","true").format('csv').load(path)
    column = df.columns
    
    for col_name in column:
        if "Date" in col_name or "date" in col_name:
            df = df.withColumn(("modified_"+col_name), to_date(col(col_name)))
    output_path = '/mnt/transformed-data/walmart/' +i+ '/'
    df.write.csv(output_path, header=True, mode="overwrite")
    

In [ ]:
customer_df=spark.read.csv('/mnt/transformed-data/walmart/customer/',inferSchema=True,header=True)
display(customer_df)
customer_df.printSchema()

customer_id,customer_name,customer_contact,date_of_birth,email,address,gender,customer_type_id,membership_to,membership_to_from,modified_date_of_birth
CTM00001,Mahesh,96548245,2000-11-23T00:00:00Z,maheshpunk@gmail.com,rohini,male,CT00S,2016-12-10T00:00:00Z,2017-12-10T00:00:00Z,2000-11-23
CTM00002,Rakesh,96545489,1998-07-13T00:00:00Z,rakeshrap@gmail.com,cannaught palace,male,CT00G,2016-05-17T00:00:00Z,2019-05-17T00:00:00Z,1998-07-13
CTM00003,Shanti,86544759,1996-10-28T00:00:00Z,shanti203@gmail.com,pitampura,female,CT00G,2015-12-05T00:00:00Z,2018-12-05T00:00:00Z,1996-10-28
CTM00004,Vimla,92512549,1994-10-11T00:00:00Z,vimlakanti@gmail.com,india gate,female,CT00P,2012-07-10T00:00:00Z,2017-07-10T00:00:00Z,1994-10-11
CTM00005,Ahmad,84824589,2000-09-11T00:00:00Z,ahmadkhan11@gmail.com,pragati maidan,male,CT00S,2016-10-15T00:00:00Z,2017-10-15T00:00:00Z,2000-09-11
CTM00006,Vikash,96618554,1998-02-13T00:00:00Z,vikashhero@gmail.com,rohini,male,CT00S,2016-11-11T00:00:00Z,2017-11-11T00:00:00Z,1998-02-13
CTM00007,kalika,87521452,2001-12-22T00:00:00Z,kalikaomg@gmail.com,dwarka,female,CT00S,2016-08-05T00:00:00Z,2017-08-05T00:00:00Z,2001-12-22
CTM00008,karan,80821452,1991-10-20T00:00:00Z,karank102@gmail.com,cannaught palace,male,CT0PR,2011-09-26T00:00:00Z,2019-09-26T00:00:00Z,1991-10-20
CTM00009,Vipul,88548887,1992-03-29T00:00:00Z,singhvipul@gmail.com,noida,male,CT0PR,2014-06-15T00:00:00Z,2022-06-15T00:00:00Z,1992-03-29
CTM00010,Akhilesh,94547212,2000-10-23T00:00:00Z,akhilesh887@gmail.com,rohini,male,CT00S,2016-12-10T00:00:00Z,2017-12-10T00:00:00Z,2000-10-23


root
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_contact: integer (nullable = true)
 |-- date_of_birth: timestamp (nullable = true)
 |-- email: string (nullable = true)
 |-- address: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- customer_type_id: string (nullable = true)
 |-- membership_to: timestamp (nullable = true)
 |-- membership_to_from: timestamp (nullable = true)
 |-- modified_date_of_birth: date (nullable = true)

